In [1]:
import gym
import numpy as np
import torch
import matplotlib.pyplot as plt
import time

In [2]:
from gym.wrappers import Monitor

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [4]:
import math
import copy
from torch.distributions import Categorical
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []
    
    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

In [6]:
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, n_latent_var):
        super(ActorCritic, self).__init__()

        # actor
        self.action_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, action_dim),
                nn.Softmax(dim=-1)
                )
        
        # critic
        self.value_layer = nn.Sequential(
                nn.Linear(state_dim, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, n_latent_var),
                nn.Tanh(),
                nn.Linear(n_latent_var, 1)
                )
    ##Changed this part    
    def forward(self, inputs):
            x = self.action_layer(inputs)
            return x
    
    def act(self, state, memory):
        state = torch.from_numpy(state).float().to(device) 
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        action = dist.sample()
        
        memory.states.append(state)
        memory.actions.append(action)
        memory.logprobs.append(dist.log_prob(action))
        
        return action.item()
    
    def evaluate(self, state, action):
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        
        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()
        
        state_value = self.value_layer(state)
        
        return action_logprobs, torch.squeeze(state_value), dist_entropy

In [7]:
class PPO:
    def __init__(self, state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.policy = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr, betas=betas)
        self.policy_old = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.MseLoss = nn.MSELoss()
    
    def update(self, memory):   
        # Monte Carlo estimate of state rewards:
        rewards = []
        surr_loss = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
        
        # Normalizing the rewards:
        rewards = torch.tensor(rewards).to(device)
        #print(rewards)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)
        #print("Rewardsd Mean: ") 
        #print(rewards.mean())
        # convert list to tensor
        old_states = torch.stack(memory.states).to(device).detach()
        old_actions = torch.stack(memory.actions).to(device).detach()
        old_logprobs = torch.stack(memory.logprobs).to(device).detach()
        
        # Optimize policy for K epochs:
        for _ in range(self.K_epochs):
            # Evaluating old actions and values :
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)
            
            # Finding the ratio (pi_theta / pi_theta__old):
            ratios = torch.exp(logprobs - old_logprobs.detach())
            print("State Values : ", state_values.detach())
            # Finding Surrogate Loss:
            advantages = rewards - state_values.detach()
            print("Advantages : ", advantages)
            
            surr1 = ratios * advantages
            #surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            #loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            #surr_loss.append(loss)
            print("Surrogate Loss: ", surr1)
            print("Surrogate Loss Mean : ", surr1.mean())
            print("Surrogate Loss Absolute Value : ", torch.abs(surr1.mean()))
            # take gradient step
            #self.optimizer.zero_grad()
            #surr1.mean().backward()
            #self.optimizer.step()
            
                  
        # Copy new weights into old policy:       
        self.policy_old.load_state_dict(self.policy.state_dict())
        #print("Surrogate Loss Mean: ",surr1.mean())
        return torch.abs(surr1.mean())

In [8]:
solved_reward = 30         # stop training if avg_reward > solved_reward
log_interval = 100           # print avg reward in the interval
max_episodes = 1000        # max training episodes
max_timesteps = 30         # max timesteps in one episode
n_latent_var = 64           # number of variables in hidden layer
update_timestep = 2000      # update policy every n timesteps
lr = 0.002
betas = (0.9, 0.999)
gamma = 0.99                # discount factor
K_epochs = 1                # update policy for K epochs
eps_clip = 0.2              # clip parameter for PPO
random_seed = None
render = False

In [10]:
env = gym.make('LunarLander-v2')

In [11]:
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
print("State Dimension ", state_dim, "\t Action Dimension ", action_dim)

State Dimension  8 	 Action Dimension  4


In [12]:
def return_random_agents(num_agents):
    
    agents = []
    for _ in range(num_agents):
        ##Change this part
        agent = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        
        for param in agent.parameters():
            param.requires_grad = False
         ##Commented this part   
        #init_weights(agent)
        agents.append(agent)
        
        
    return agents

#### I tried to change this function

In [13]:
def run_agents(agents):
    
    reward_agents = []
    total_loss = []
    env_name = "LunarLander-v2"
    env = gym.make('LunarLander-v2')
    if random_seed:
        torch.manual_seed(random_seed)
        env.seed(random_seed)
    
    memory = Memory()
    ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip)
    print(lr,betas)
    
    # logging variables
    running_reward = 0
    avg_length = 0
    timestep = 0
    surri=0
    s1 = []
    s2 = 0
    s_temp = []
    surr_actual = []
    
    # training loop
    #for agent in agents:
    for i_episode in range(1, max_episodes+1):
        #agent.eval()
        state = env.reset()
        for t in range(max_timesteps):
            timestep += 1
            
            # Running policy_old:
            action = ppo.policy_old.act(state, memory)
            state, reward, done, _ = env.step(action)
            
            # Saving reward and is_terminal:
            memory.rewards.append(reward)
            memory.is_terminals.append(done)
            
            # update if its time
            if timestep % update_timestep == 0:
                surri=ppo.update(memory)
                #print("Returned Surrogate Loss ", surri)
                memory.clear_memory()
                timestep = 0
            
            running_reward += reward
            if render:
                env.render()
            if done:
                break
                
        avg_length += t
        
        # stop training if avg_reward > solved_reward
        if running_reward > (log_interval*solved_reward):
            print("########## Solved! ##########")
            torch.save(ppo.policy.state_dict(), './PPO_{}.pth'.format(env_name))
            break
            
        # logging
        if i_episode % log_interval == 0:
            avg_length = int(avg_length/log_interval)
            running_reward = int((running_reward/log_interval))
            print(' Episode {} '.format(i_episode))
            running_reward = 0
            avg_length = 0
        s1.append(surri)
        #surr_mean = torch.mean(torch.stack(surr_actual))    
    s1 = [i for i in s1 if i != 0]
    #print(torch.mean(torch.stack(s1)))
    #print("\n")
            
    #print(total_loss)
    return torch.mean(torch.stack(s1))

In [14]:
def return_average_score(agent, runs):
    #score = 0.
    #for i in range(runs):
    #print(run_agents([agent]))
    score = run_agents([agent])
    return score#/runs

#### <--Till Now Only worked -->

In [15]:
def run_agents_n_times(agents, runs):
    avg_score = []
    for agent in agents:
        avg_score.append(return_average_score(agent,runs))
    return avg_score

In [16]:
def mutate(agent):

    child_agent = copy.deepcopy(agent)
    
    mutation_power = 0.02 #hyper-parameter, set from https://arxiv.org/pdf/1712.06567.pdf
            
    for param in child_agent.parameters():
    
        if(len(param.shape)==4): #weights of Conv2D

            for i0 in range(param.shape[0]):
                for i1 in range(param.shape[1]):
                    for i2 in range(param.shape[2]):
                        for i3 in range(param.shape[3]):
                            
                            param[i0][i1][i2][i3]+= mutation_power * np.random.randn()
                                
                                    

        elif(len(param.shape)==2): #weights of linear layer
            for i0 in range(param.shape[0]):
                for i1 in range(param.shape[1]):
                    
                    param[i0][i1]+= mutation_power * np.random.randn()
                        

        elif(len(param.shape)==1): #biases of linear layer or conv layer
            for i0 in range(param.shape[0]):
                
                param[i0]+=mutation_power * np.random.randn()

    return child_agent

In [17]:
def return_children(agents, sorted_parent_indexes, elite_index):
    
    children_agents = []
    
    #first take selected parents from sorted_parent_indexes and generate N-1 children
    for i in range(len(agents)-1):
        
        selected_agent_index = sorted_parent_indexes[np.random.randint(len(sorted_parent_indexes))]
        children_agents.append(mutate(agents[selected_agent_index]))

    #now add one elite
    elite_child = add_elite(agents, sorted_parent_indexes, elite_index)
    children_agents.append(elite_child)
    elite_index=len(children_agents)-1 #it is the last one
    
    return children_agents, elite_index

In [18]:
def add_elite(agents, sorted_parent_indexes, elite_index=None, only_consider_top_n=10):
    
    candidate_elite_index = sorted_parent_indexes[:only_consider_top_n]
    
    if(elite_index is not None):
        candidate_elite_index = np.append(candidate_elite_index,[elite_index])
        
    top_score = None
    top_elite_index = None
    
    for i in candidate_elite_index:
        score = return_average_score(agents[i],runs=5)
        print("Score for elite i ", i, " is ", score)
        
        if(top_score is None):
            top_score = score
            top_elite_index = i
        elif(score > top_score):
            top_score = score
            top_elite_index = i
            
    print("Elite selected with index ",top_elite_index, " and score", top_score)
    
    child_agent = copy.deepcopy(agents[top_elite_index])
    return child_agent
    

In [19]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [20]:
game_actions = 2 #2 actions possible: left or right

#disable gradients as we will not use them
#torch.set_grad_enabled(False)

# initialize N number of agents
num_agents = 10
agents = return_random_agents(num_agents)

# How many top agents to consider as parents
top_limit = 3

# run evolution until X generations
generations = 5

elite_index = None

for generation in range(generations):

    # return rewards of agents
    losses = run_agents_n_times(agents, 1) #return average of 3 runs

    # sort by rewards
    sorted_parent_indexes = np.argsort(losses)[::-1][:top_limit]#reverses and gives top values (argsort sorts by ascending by default) https://stackoverflow.com/questions/16486252/is-it-possible-to-use-argsort-in-descending-order
    print("\n Sorting Parent Indexes: ",sorted_parent_indexes)
    print(" Data Type: ", type(sorted_parent_indexes))
    print("Sorting Completed")
    print("Selecting Top Parents")
    
    top_losses = []
    for best_parent in sorted_parent_indexes:
        top_losses.append(losses[best_parent])
   
    print("\n \n Generation ", generation, " | Mean Losses: ", torch.mean(torch.stack(losses)), " | Mean of top 5: ",torch.mean(torch.stack(top_losses[:5])))
    
    print("Top ",top_limit," scores", sorted_parent_indexes)
    print("Rewards for top: ",top_losses)
    
    # setup an empty list for containing children agents
    children_agents, elite_index = return_children(agents, sorted_parent_indexes, elite_index)

    # kill all agents, and replace them with their children
    agents = children_agents

0.002 (0.9, 0.999)
State Values :  tensor([-0.0111, -0.0111, -0.0115,  ..., -0.0194, -0.0214, -0.0215])
Advantages :  tensor([0.0808, 0.0804, 0.0756,  ..., 2.3533, 2.3395, 2.3178])
Surrogate Loss:  tensor([0.0808, 0.0804, 0.0756,  ..., 2.3533, 2.3395, 2.3178],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(0.0135, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.0135, grad_fn=<AbsBackward>)
 Episode 100 
State Values :  tensor([-0.0233, -0.0257, -0.0230,  ..., -0.0164, -0.0129, -0.0123])
Advantages :  tensor([0.4348, 0.3742, 0.4090,  ..., 2.7026, 2.6727, 2.7633])
Surrogate Loss:  tensor([0.4348, 0.3742, 0.4090,  ..., 2.7026, 2.6727, 2.7633],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(0.0129, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.0129, grad_fn=<AbsBackward>)
State Values :  tensor([-0.0111, -0.0130, -0.0118,  ..., -0.0213, -0.0210, -0.0228])
Advantages :  tensor([-0.5446, -0.5164, -0.3781,  ...,  2.3196,  2.3

State Values :  tensor([-0.0667, -0.0678, -0.0643,  ..., -0.0931, -0.0958, -0.0948])
Advantages :  tensor([0.1460, 0.1795, 0.1123,  ..., 2.6977, 2.8119, 2.8366])
Surrogate Loss:  tensor([0.1460, 0.1795, 0.1123,  ..., 2.6977, 2.8119, 2.8366],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(0.0497, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.0497, grad_fn=<AbsBackward>)
 Episode 400 
State Values :  tensor([-0.0876, -0.0868, -0.0839,  ..., -0.0468, -0.0438, -0.0421])
Advantages :  tensor([-1.1879, -1.1632, -1.2350,  ...,  2.6477,  2.7025,  2.8264])
Surrogate Loss:  tensor([-1.1879, -1.1632, -1.2350,  ...,  2.6477,  2.7025,  2.8264],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(0.0434, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.0434, grad_fn=<AbsBackward>)
 Episode 500 
State Values :  tensor([-0.0405, -0.0391, -0.0368,  ..., -0.0162, -0.0174, -0.0176])
Advantages :  tensor([-0.3254, -0.2207, -0.1510,  ...,  3.268

 Episode 700 
State Values :  tensor([0.2635, 0.2660, 0.2655,  ..., 0.3650, 0.3680, 0.3661])
Advantages :  tensor([0.5159, 0.5286, 0.5683,  ..., 3.4035, 3.4809, 3.5304])
Surrogate Loss:  tensor([0.5159, 0.5286, 0.5683,  ..., 3.4035, 3.4809, 3.5304],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(-0.3059, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.3059, grad_fn=<AbsBackward>)
State Values :  tensor([0.3648, 0.3651, 0.3655,  ..., 0.3065, 0.3055, 0.3064])
Advantages :  tensor([-2.0009, -2.0288, -2.0536,  ...,  2.5731,  2.6869,  2.6705])
Surrogate Loss:  tensor([-2.0009, -2.0288, -2.0536,  ...,  2.5731,  2.6869,  2.6705],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(-0.3085, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.3085, grad_fn=<AbsBackward>)
 Episode 800 
State Values :  tensor([0.2151, 0.2126, 0.2089,  ..., 0.3035, 0.3059, 0.3057])
Advantages :  tensor([-2.5013, -2.5682, -2.6220,  ...,  2.5054,  2.6014,  2.6

State Values :  tensor([0.0618, 0.0619, 0.0655,  ..., 0.1205, 0.1187, 0.1200])
Advantages :  tensor([-0.4341, -0.3304, -0.3991,  ...,  2.6977,  2.6380,  2.6698])
Surrogate Loss:  tensor([-0.4341, -0.3304, -0.3991,  ...,  2.6977,  2.6380,  2.6698],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(-0.1276, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.1276, grad_fn=<AbsBackward>)
 Episode 100 
State Values :  tensor([0.1200, 0.1185, 0.1169,  ..., 0.0957, 0.0958, 0.0962])
Advantages :  tensor([-0.7817, -0.7398, -0.6903,  ...,  2.4585,  2.4748,  2.4938])
Surrogate Loss:  tensor([-0.7817, -0.7398, -0.6903,  ...,  2.4585,  2.4748,  2.4938],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(-0.1314, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.1314, grad_fn=<AbsBackward>)
State Values :  tensor([0.0944, 0.0924, 0.0924,  ..., 0.1751, 0.1741, 0.1751])
Advantages :  tensor([-0.9317, -0.7696, -0.7673,  ...,  2.3232,  2.4028,  2.448

State Values :  tensor([-0.0109, -0.0133, -0.0172,  ..., -0.0955, -0.0956, -0.0927])
Advantages :  tensor([-1.1471, -1.1849, -1.0843,  ...,  3.7521,  3.8743,  3.9608])
Surrogate Loss:  tensor([-1.1471, -1.1849, -1.0843,  ...,  3.7521,  3.8743,  3.9608],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(0.0422, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.0422, grad_fn=<AbsBackward>)
 Episode 400 
State Values :  tensor([0.0226, 0.0244, 0.0299,  ..., 0.0085, 0.0144, 0.0202])
Advantages :  tensor([-1.4250, -1.4245, -1.3699,  ...,  3.8098,  3.8730,  3.9460])
Surrogate Loss:  tensor([-1.4250, -1.4245, -1.3699,  ...,  3.8098,  3.8730,  3.9460],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(0.0267, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.0267, grad_fn=<AbsBackward>)
 Episode 500 
State Values :  tensor([ 0.0224,  0.0191,  0.0252,  ..., -0.0693, -0.0707, -0.0680])
Advantages :  tensor([0.4856, 0.2541, 0.3030,  ..., 4.2

...

State Values :  tensor([0.0682, 0.0704, 0.0703,  ..., 0.0475, 0.0471, 0.0483])
Advantages :  tensor([-1.8662, -1.8792, -1.9116,  ...,  2.3371,  2.4187,  2.2792])
Surrogate Loss:  tensor([-1.8662, -1.8792, -1.9116,  ...,  2.3371,  2.4187,  2.2792],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(-0.0557, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.0557, grad_fn=<AbsBackward>)
 Episode 600 
State Values :  tensor([0.0534, 0.0535, 0.0519,  ..., 0.0274, 0.0252, 0.0227])
Advantages :  tensor([0.6979, 0.6661, 0.6789,  ..., 3.6443, 3.7589, 3.8538])
Surrogate Loss:  tensor([0.6979, 0.6661, 0.6789,  ..., 3.6443, 3.7589, 3.8538],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(-0.0553, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.0553, grad_fn=<AbsBackward>)
 Episode 700 
State Values :  tensor([0.0203, 0.0173, 0.0147,  ..., 0.0747, 0.0738, 0.0722])
Advantages :  tensor([-0.1081, -0.0636, -0.0044,  ...,  3.6177,  3.5722,  3.4

 Episode 900 
State Values :  tensor([0.0257, 0.0274, 0.0252,  ..., 0.0278, 0.0293, 0.0344])
Advantages :  tensor([-0.6005, -0.5709, -0.5107,  ...,  2.6391,  2.7249,  2.8184])
Surrogate Loss:  tensor([-0.6005, -0.5709, -0.5107,  ...,  2.6391,  2.7249,  2.8184],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(-0.0225, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.0225, grad_fn=<AbsBackward>)
State Values :  tensor([0.0360, 0.0411, 0.0428,  ..., 0.0570, 0.0611, 0.0592])
Advantages :  tensor([0.0570, 0.1592, 0.2524,  ..., 2.6415, 2.7894, 2.9359])
Surrogate Loss:  tensor([0.0570, 0.1592, 0.2524,  ..., 2.6415, 2.7894, 2.9359],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(-0.0221, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.0221, grad_fn=<AbsBackward>)
 Episode 1000 

 Sorting Parent Indexes:  [5 1 7]
 Data Type:  <class 'numpy.ndarray'>
Sorting Completed
Selecting Top Parents

 
 Generation  4  | Mean Losses:  tensor(0

State Values :  tensor([-0.0261, -0.0235, -0.0232,  ..., -0.0264, -0.0267, -0.0275])
Advantages :  tensor([1.0333, 1.1300, 1.2462,  ..., 3.2885, 3.4335, 3.6633])
Surrogate Loss:  tensor([1.0333, 1.1300, 1.2462,  ..., 3.2885, 3.4335, 3.6633],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(0.0253, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.0253, grad_fn=<AbsBackward>)
 Episode 200 
State Values :  tensor([-0.0461, -0.0479, -0.0486,  ..., -0.0180, -0.0179, -0.0179])
Advantages :  tensor([-0.2421, -0.2792, -0.1587,  ...,  2.6310,  2.7346,  2.8514])
Surrogate Loss:  tensor([-0.2421, -0.2792, -0.1587,  ...,  2.6310,  2.7346,  2.8514],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(0.0239, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.0239, grad_fn=<AbsBackward>)
 Episode 300 
State Values :  tensor([-0.0157, -0.0157, -0.0161,  ...,  0.0022,  0.0027,  0.0031])
Advantages :  tensor([-2.5520, -2.4959, -2.4480,  ...,  1.659

 Episode 500 
State Values :  tensor([-0.2545, -0.2536, -0.2527,  ..., -0.2507, -0.2497, -0.2488])
Advantages :  tensor([-0.6963, -0.6686, -0.6201,  ...,  4.1717,  4.0250,  3.8140])
Surrogate Loss:  tensor([-0.6963, -0.6686, -0.6201,  ...,  4.1717,  4.0250,  3.8140],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(0.2464, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.2464, grad_fn=<AbsBackward>)
State Values :  tensor([-0.2483, -0.2475, -0.2469,  ..., -0.2066, -0.2047, -0.2028])
Advantages :  tensor([0.4366, 0.4590, 0.2726,  ..., 2.9746, 3.0485, 3.1032])
Surrogate Loss:  tensor([0.4366, 0.4590, 0.2726,  ..., 2.9746, 3.0485, 3.1032],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(0.2471, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.2471, grad_fn=<AbsBackward>)
 Episode 600 
State Values :  tensor([-0.2474, -0.2465, -0.2457,  ..., -0.2132, -0.2115, -0.2095])
Advantages :  tensor([0.5514, 0.6054, 0.6342,  ..., 3.2362, 3

State Values :  tensor([-0.0335, -0.0316, -0.0323,  ...,  0.0031,  0.0029,  0.0046])
Advantages :  tensor([0.2752, 0.1651, 0.2049,  ..., 2.8553, 2.7118, 2.8004])
Surrogate Loss:  tensor([0.2752, 0.1651, 0.2049,  ..., 2.8553, 2.7118, 2.8004],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(0.0161, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.0161, grad_fn=<AbsBackward>)
 Episode 900 
State Values :  tensor([ 0.0063,  0.0079,  0.0089,  ..., -0.0033, -0.0036, -0.0035])
Advantages :  tensor([-1.0151, -0.9535, -1.0442,  ...,  3.6519,  3.5529,  3.6160])
Surrogate Loss:  tensor([-1.0151, -0.9535, -1.0442,  ...,  3.6519,  3.5529,  3.6160],
       grad_fn=<MulBackward0>)
Surrogate Loss Mean :  tensor(0.0139, grad_fn=<MeanBackward0>)
Surrogate Loss Absolute Value :  tensor(0.0139, grad_fn=<AbsBackward>)
State Values :  tensor([-0.0030, -0.0029, -0.0021,  ..., -0.0456, -0.0436, -0.0455])
Advantages :  tensor([1.4815, 1.5096, 1.5535,  ..., 2.9079, 2.9302, 3.2122]

In [ ]:
def play_agent(agent):
    env = gym.make("LunarLander-v2")
    env_record = Monitor(env, './video', force=True)
    observation = env_record.reset()
    last_observation = observation
    r=0
    memory = Memory()
    ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip)
    for _ in range(250):
        env_record.render()
        action = ppo.policy_old.act(observation, memory)
        new_observation, reward, done, _ = env_record.step(action)
        r=r+reward
        observation = new_observation

        if(done):
            break
    env_record.close()
    print("Rewards: ",r)      

In [ ]:
play_agent(agents[1])